In [1]:
import pandas as pd
import numpy as np
import datetime
import geopandas as gpd

***
__Preparar dados no formato exigido pelo SaTScan para varredura permutação espaço-tempo ou poisson discreto: população, casos e coordenadas.__
- PA
- PA/MA/TO
- BR
***

## Casos
The expected format of the case file, using the Poisson probability model is:

`<Location ID><Number of Cases><Date/Time><Covariate 1><Covariate N>`

In [2]:
df = pd.read_csv('../data/indicadores/visceral/indicador_14_total_casos_novos_municipio_infeccao.csv')

para = df.loc[df.CO_MN_INF.astype('string').str.startswith('15'), :].copy()

pa_ma_to = df.loc[
    df.CO_MN_INF.astype('string').str.startswith('15') |
    df.CO_MN_INF.astype('string').str.startswith('17') |
    df.CO_MN_INF.astype('string').str.startswith('21'),
    :
].copy()

In [3]:
def casos_satscan(df):
    casos = pd.DataFrame()
    for it in df.columns[1:]:
        temp = df.loc[:, ['CO_MN_INF', it]].copy()
        temp.columns = ['location_id', 'cases']
        temp['time'] = it
        casos = pd.concat([casos, temp])
    casos.cases = casos.cases.astype(int)
    return casos
    
casos_satscan(df).to_csv('../output/satscan_experiment_data/casos_BR.csv', index=False)
casos_satscan(para).to_csv('../output/satscan_experiment_data/casos_PA.csv', index=False)
casos_satscan(pa_ma_to).to_csv('../output/satscan_experiment_data/casos_PA_MA_TO.csv', index=False)
del df
del para
del pa_ma_to

## População
The expected format of the population file is:

`<Location ID> <Date/Time> <Population> <Covariate 1> ... <Covariate N>`


In [4]:
pop = pd.read_csv('../data/processed/processed_populacao.csv')

#há municípios sem informação populacional: preencher com média
index = pop.loc[pop.T.isnull().any(), :].index
display(pop.loc[pop.T.isnull().any(), :])

for it in index:
    pop.iloc[it, :] = pop.iloc[it, :].fillna(pop.iloc[it, 1:].mean())

pop.MUNIC_RES = pop.MUNIC_RES.astype(int)
display(pop.loc[pop.T.isnull().any(), :])

pop_pa = pop.loc[pop.MUNIC_RES.astype('string').str.startswith('15'), :].copy()
pop_pa_ma_to = pop.loc[
    pop.MUNIC_RES.astype('string').str.startswith('15') |
    pop.MUNIC_RES.astype('string').str.startswith('17') |
    pop.MUNIC_RES.astype('string').str.startswith('21'), 
    :
].copy()

,MUNIC_RES,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
224,150475,NaN,NaN,NaN,NaN,NaN,15018.0,15232.0,15341.0,15446.0,15548.0,15646.0,15982.0,16084.0
803,220672,NaN,8501.0,7895.0,8039.0,8149.0,8227.0,8289.0,8330.0,8366.0,8398.0,8425.0,8536.0,8570.0
4503,421265,NaN,NaN,NaN,NaN,NaN,9416.0,9687.0,9761.0,9835.0,9908.0,9980.0,10022.0,10091.0
4605,422000,NaN,NaN,NaN,NaN,NaN,11136.0,11628.0,11824.0,12018.0,12212.0,12403.0,12570.0,12760.0
4923,431454,NaN,NaN,NaN,NaN,NaN,2578.0,2681.0,2800.0,2824.0,2847.0,2868.0,2968.0,3003.0
5160,500627,NaN,NaN,NaN,NaN,NaN,4723.0,4942.0,5047.0,5150.0,5251.0,5350.0,5455.0,5555.0


,MUNIC_RES,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019


In [5]:
def pop_satscan(df):
    pop = pd.DataFrame()
    for it in df.columns[1:]:
        temp = df.loc[:, ['MUNIC_RES', it]].copy()
        temp.columns = ['location_id', 'pop']    
        temp.insert(loc=1, column='time', value=it)
        pop = pd.concat([pop, temp])
    return pop

pop_satscan(pop).to_csv('../experimentos/satscan_experiment_data/pop_BR.csv', index=False)
pop_satscan(pop_pa).to_csv('../experimentos/satscan_experiment_data/pop_PA.csv', index=False)
pop_satscan(pop_pa_ma_to).to_csv('../experimentos/satscan_experiment_data/pop_PA_MA_TO.csv', index=False)
del pop
del pop_pa
del pop_pa_ma_to

## Coordenadas

In [6]:
geo = gpd.read_file('../data/raw/poligonos/BR_Municipios_2020.shp')

In [7]:
geo.CD_MUN = geo.CD_MUN.astype('string').str[0:6]
geo.to_file('../experimentos/satscan_experiment_data/geo_br')
geo.loc[geo.CD_MUN.str.startswith('15'), :].to_file('../experimentos/satscan_experiment_data/geo_para')
geo.loc[
    geo.CD_MUN.str.startswith('15') |
    geo.CD_MUN.str.startswith('17') |
    geo.CD_MUN.str.startswith('21'), 
    :
].to_file('../experimentos/satscan_experiment_data/geo_pa_ma_to')
del geo

## Arquivo de Grade

> "O arquivo de grade opcional define os centróides dos círculos utilizados pela estatística de varredura. Se
não for especificado nenhum arquivo de grade, as coordenadas dadas no arquivo de coordenadas serão
usadas para esta finalidade."

Existem no Pará municípios muito extensos. Se o arquivo de grade não for especificado o centro do município ficará como padrão e distante das regiões densamente habitadas (cidade), por exemplo Altamira. O arquivo de grade vai definir como centróide do município a cidade deste. 

the expected format of grid file is:

`<latitude><longitude`


In [8]:
geo = gpd.read_file('../data/raw/poligonos/BR_Localidades_2010_v1.shp')

#Em 2010 Mujuí dos Campos era Vila.
display(
    geo.loc[
        (geo.NM_UF == 'PARÁ') & 
        (geo.NM_CATEGOR == 'CIDADE') | 
        (geo.NM_LOCALID == 'MUJUÍ DOS CAMPOS'),
        ['NM_LOCALID', 'LONG', 'LAT', 'ALT', 'GMRotation', 'geometry']
    ]
)

,NM_LOCALID,LONG,LAT,ALT,GMRotation,geometry
1050,ABAETETUBA,-48.884404,-1.723470,10.120343,0.0,POINT (-48.88440 -1.72347)
1052,ABEL FIGUEIREDO,-48.396762,-4.951391,192.756651,0.0,POINT (-48.39676 -4.95139)
1053,ACARÁ,-48.200456,-1.963437,7.395031,0.0,POINT (-48.20046 -1.96344)
1058,AFUÁ,-50.391428,-0.159172,3.136401,0.0,POINT (-50.39143 -0.15917)
1060,ÁGUA AZUL DO NORTE,-50.485275,-6.804949,296.299435,0.0,POINT (-50.48527 -6.80495)
...,...,...,...,...,...,...
1847,URUARÁ,-53.737031,-3.706529,162.216302,0.0,POINT (-53.73703 -3.70653)
1854,VIGIA,-48.141499,-0.855533,7.545379,0.0,POINT (-48.14150 -0.85553)
1863,VISEU,-46.138765,-1.203279,7.618306,0.0,POINT (-46.13877 -1.20328)
1887,VITÓRIA DO XINGU,-52.011260,-2.883766,16.543706,0.0,POINT (-52.01126 -2.88377)


In [9]:
geo.loc[
    (geo.NM_UF == 'PARÁ') & 
    (geo.NM_CATEGOR == 'CIDADE') | 
    (geo.NM_LOCALID == 'MUJUÍ DOS CAMPOS'),
    ['LAT', 'LONG']

].to_csv('../experimentos/satscan_experiment_data/grid_PA.csv', index=False)

In [10]:
geo.loc[
    (geo.NM_UF.isin(['PARÁ', 'MARANHÃO', 'TOCANTINS'])) & 
    (geo.NM_CATEGOR == 'CIDADE') | 
    (geo.NM_LOCALID == 'MUJUÍ DOS CAMPOS'),
    ['LAT', 'LONG']

].to_csv('../experimentos/satscan_experiment_data/grid_PA_MA_TO.csv', index=False)

In [11]:
# se tornaram cidades depois de 2010: 
# 'PESCARIA BRAVA', 'BALNEÁRIO RINCÃO', 'MUJUÍ DOS CAMPOS', 'PINTO BANDEIRA', 'PARAÍSO DAS ÁGUAS'
geo.loc[
    (geo.NM_CATEGOR == 'CIDADE') | 
    geo.NM_LOCALID.isin(['PESCARIA BRAVA', 'BALNEÁRIO RINCÃO', 'MUJUÍ DOS CAMPOS', 'PINTO BANDEIRA', 'PARAÍSO DAS ÁGUAS']),
    ['LAT', 'LONG']
].to_csv('../experimentos/satscan_experiment_data/grid_BR.csv', index=False)